In [5]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model

In [7]:
df_sample = pd.read_csv('C://Users//snuist//Data//Lending_Club_Data_Subsample.csv', engine = 'python')

In [9]:
df_X = df_sample.iloc[:, 13:19].as_matrix()
df_y = df_sample[['bad']].as_matrix()

In [10]:
df_X.shape

(1000, 6)

In [11]:
df_y.shape

(1000, 1)

In [12]:
# 변수가 뭐였는지 까먹지 않도록 하는 것
X_varnames = df_sample.columns[13:19]
X_varnames

Index(['loan_amnt', 'installment', 'annual_inc', 'dti', 'delinq_2yrs',
       'inq_last_6mths'],
      dtype='object')

In [17]:
# OLS 로 fitting 시킨 것
mdl_OLS = linear_model.LinearRegression()
mdl_OLS.fit(df_X, df_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [18]:
mdl_OLS.coef_

array([[  5.43612722e-06,  -1.39347809e-04,  -2.74818631e-07,
          3.03875895e-03,  -1.00213563e-02,   2.48954400e-02]])

In [20]:
# Adaptive Lasso에서는 설명력이 약한 변수들에 가중을 크게 주어 페날티를 크게 주려고 한다.
# 각 베타를 OLS를 통해 추정된 절대값 틸다베타로 나누어준 것을 Bj라고 할 때, 
# 그것이 (Yi - {X}b)^2 이런 모양이 되기 위해서는 X는 틸다베타와 기존 Xij를 곱해줘야 한다. 
# 여기의 weights는 기존 Xij에 곱해지는 절대값 틸다
weights = np.abs(mdl_OLS.coef_)
weights

array([[  5.43612722e-06,   1.39347809e-04,   2.74818631e-07,
          3.03875895e-03,   1.00213563e-02,   2.48954400e-02]])

In [25]:
# intercept를 없애고 y-beta*x꼴로 나타내어 계산의 편의를 도모하기 위해 평균을 빼서 중심을 0으로 만들어버린다.
df_X_0 = df_X - np.mean(df_X, axis = 0)
np.mean(df_X_0, axis = 0)

array([  6.98491931e-13,  -1.90993887e-14,   2.32830644e-12,
        -2.84217094e-16,   6.21724894e-18,   8.17124146e-17])

In [26]:
# x에서 0으로 맞춰주는 김에 여기도 중심을 0으로 맞춰주기
df_y_0 = df_y - np.mean(df_y, axis = 0)

In [32]:
# Weight가 곱해진 변형된 Xi
X_w = df_X_0 * weights
alpha = 10**(-4)

#Intercept없는 라쏘모델을 불러올 것임
clf = linear_model.Lasso(alpha = alpha, fit_intercept = False)
clf.fit(X_w, df_y_0)

print(clf.coef_)

# 치환해서 풀어 Bj를 구했으니까 원래 구하고자 했던 베타를 구하기 위해서 절대값 틸다베타를 곱한다. 
final_coef = clf.coef_ * weights
print(final_coef)

[  3.49164576e-04   0.00000000e+00  -4.24433174e-01   8.90530620e-01
  -0.00000000e+00   8.70726902e-01]
[[  1.89810306e-09   0.00000000e+00  -1.16642144e-07   2.70610790e-03
   -0.00000000e+00   2.16771293e-02]]


In [28]:
weights.shape

(1, 6)

In [30]:
df_X_0.shape

(1000, 6)

In [33]:
# Elastic net 불러오기 (l1_ratio와 l2_ratio의 비율을 정함으로써 penalty function을 다듬는 것)
mdl_ela = linear_model.ElasticNet(alpha= 0.1, l1_ratio = 0.5)
mdl_ela.fit(df_X, df_y)
mdl_ela.coef_

array([  1.35580201e-06,  -2.21371458e-05,  -2.76536184e-07,
         2.29246381e-03,  -0.00000000e+00,   0.00000000e+00])

In [34]:
# 여기서 alpha라고 되어있는 것은 람다이다.
# 1l_ratio를 크게 하는 것은 kink되어 있는 l1 norm과 더 유사하게 만든다는 것이다. 
# 즉, 많은 것이 0값을 띄며 탈락될 것이다. 실제로 위의 0.5인 경우보다 더 많은 변수가 탈락되었다. 
mdl_ela = linear_model.ElasticNet(alpha= 0.1, l1_ratio = 0.99)
mdl_ela.fit(df_X, df_y)
mdl_ela.coef_

array([  7.25749782e-07,  -0.00000000e+00,  -3.09882849e-07,
         1.37272631e-03,  -0.00000000e+00,   0.00000000e+00])